# Conversion Window 정의
- **정의**: 전환 과정 시작 후 구매/전환까지 시간을 측정하는 기간. 사용자 행동과 전환율 분석에 사용됨.
- **예시**:
  - **온라인 쇼핑**: 제품 처음 본 순간부터 구매 완료까지 시간 추적.
  - **앱 사용**: 앱 다운로드 후 첫 사용 또는 특정 기능 사용까지 시간 분석.
  - **서비스 가입**: 서비스 가입 후 실제 사용 시작까지 걸리는 시간 측정.
  - **온라인 광고**: 광고 클릭 후 실제 구매 또는 목표 전환 행동까지 시간 분석.
- **완전성과 적시성**:
  - 완전성 :  '완전성(Completeness)'은 수집된 데이터가 전체적인 사용자 행동을 얼마나 잘 대표하는지.
  - 적시성 : '적시성(Timeliness)'은 분석된 정보가 얼마나 신속하게 사용 가능한지.
  
  - 예시
    - **짧은 Conversion Window**: 빠른 데이터 분석 가능, 전체 행동 패턴 미반영 가능성.
      - 적시성 up, 완전성 down
    - **긴 Conversion Window**: 분석의 적시성 저하 가능성. 길이 설정은 비즈니스 목표와 제품 특성에 따라 달라짐.
      - 적시성 down, 완전성 up



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv ('/content/drive/MyDrive/메타코드/데이터분석_연습/conversion.csv의 사본' )
# pay at 빈경우는 전환이 발생하지 않는 경우임
df.head()

,transaction_id,apply_at,pay_at
0,2560,2023-01-01,2023-01-08
1,2844,2023-01-01,2023-01-06
2,1878,2023-01-01,NaN
3,2374,2023-01-01,2023-01-02
4,3923,2023-01-01,2023-01-08


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  2170 non-null   int64 
 1   apply_at        2170 non-null   object
 2   pay_at          1062 non-null   object
dtypes: int64(1), object(2)
memory usage: 51.0+ KB


In [51]:
# time 형태로 변경
df["apply_at"] = pd.to_datetime(df.apply_at)
df["pay_at"] = pd.to_datetime(df.pay_at)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  2170 non-null   int64         
 1   apply_at        2170 non-null   datetime64[ns]
 2   pay_at          665 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 51.0 KB


In [53]:
# apply 와 pay의 시간 분포
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta

In [54]:
# 객체의 총 시간을 초 단위로 반환
df["td"] = (df.pay_at - df.apply_at).dt.total_seconds()/ (3600*24) #  daily

In [55]:
# 확인
df.head()

,transaction_id,apply_at,pay_at,td
0,2560,2023-01-01,2023-01-08,7.0
1,2844,2023-01-01,2023-01-06,5.0
2,1878,2023-01-01,NaT,NaN
3,2374,2023-01-01,2023-01-02,1.0
4,3923,2023-01-01,2023-01-08,7.0


In [56]:
# percentile : 데이터 분포, 특정 비율에 해당하는 값을 나타냄
# 데이터 세트 내에서 이 값보다 낮은 값들의 비율을 의미한다.
lis = np.array([1,2,3,4,5,6,7,8,9, 100])
np.percentile(lis, 50)


5.5

In [57]:
thread = 4
# 전환이 일어난 수
# 분모로 잡는다.
a = df[~df.pay_at.isna()].shape[0]

# 분자
b = df[df.td <= thread].shape[0]

ratio = b/a
# 분포를 보기 위해서 histogram
fig = px.histogram(
    df
    , x=df.td
)

# 상자를 생성해서 영역표시
# (예를 들면) 10일 이내의 전환만 지표로 포함시킨다
# 시간이 지남에 따라 트렌트의 해석이 어려움이 있기 떄문에 일정 기준을 잡는 것 이것이 conversion window
# 분포가 밀집한 부분을 잡은 것
fig.add_vrect(x0=1, x1=thread, fillcolor="red", opacity=0.2)
fig.add_annotation(x = thread, text= f"CW :{thread}일 -> 전환의 {ratio:.2f} % 포함")

# add perc
lis = df[~df.td.isna()].td
# perc = 10
y1 = 100
for perc in [10,20,30,40,50,60,70,80,90]:
    dates = np.percentile(lis, perc)
    fig.add_vline(x=dates)
    fig.add_annotation(x=dates, y=y1, text =f"precentile: {perc}% -> {dates}일")
    y1 -= 10
fig.show()

# Exercise
- 일시정지 후 스스로 문제를 풀어보고 강의를 수강해 주세요

[문제]
  - conversion.csv 파일에 저장된 데이터를 활용하여, Conversion Window(CW)를 각각 1일과 10일로 설정하고, 이에 따른 데이터 변화를 시각화하세요.
  - 분석시점이 23/01/27일 23:59:59 라고 가정하고 데이터를 아래의 조건으로 필터링 하시오

In [93]:
import pandas as pd
df = pd.read_csv ('/content/drive/MyDrive/메타코드/데이터분석_연습/conversion.csv의 사본' )
df['apply_at'] = pd.to_datetime(df.apply_at)
df['pay_at'] = pd.to_datetime(df.pay_at)

df['pay_at'] = pd.to_datetime(np.where ( df.pay_at >= pd.to_datetime('2023-01-28'),None,df.pay_at))
df.head()

,transaction_id,apply_at,pay_at
0,2560,2023-01-01,2023-01-08
1,2844,2023-01-01,2023-01-06
2,1878,2023-01-01,NaT
3,2374,2023-01-01,2023-01-02
4,3923,2023-01-01,2023-01-08


In [94]:
df["td"] = (df.pay_at - df.apply_at).dt.total_seconds() / (3600*24)

In [95]:
# 신청수
apply_cnt = df.groupby(by="apply_at").size().reset_index()
apply_cnt
conv1_cnt = df[df.td <= 1].groupby(by="apply_at").size().reset_index()
conv1_cnt
conv10_cnt = df[df.td <= 10].groupby(by="apply_at").size().reset_index()
conv10_cnt

,apply_at,0
0,2023-01-01,28
1,2023-01-02,21
2,2023-01-03,17
3,2023-01-04,33
4,2023-01-05,18
5,2023-01-06,30
6,2023-01-07,24
7,2023-01-08,33
8,2023-01-09,25
9,2023-01-10,27


In [96]:
df = pd.merge(apply_cnt, conv1_cnt, how="left", left_on="apply_at", right_on="apply_at").rename(
    columns = {
        "0_x": "apply_cnt"
        , "0_y": "conv1_cnt"
    }
)
df.head()

,apply_at,apply_cnt,conv1_cnt
0,2023-01-01,72,5.0
1,2023-01-02,67,5.0
2,2023-01-03,51,6.0
3,2023-01-04,89,9.0
4,2023-01-05,67,6.0


In [97]:
df = pd.merge(df, conv10_cnt, how="left", left_on="apply_at", right_on="apply_at").rename(
    columns = {
        0: "conv10_cnt"
    }
)
df.head()

,apply_at,apply_cnt,conv1_cnt,conv10_cnt
0,2023-01-01,72,5.0,28.0
1,2023-01-02,67,5.0,21.0
2,2023-01-03,51,6.0,17.0
3,2023-01-04,89,9.0,33.0
4,2023-01-05,67,6.0,18.0


In [98]:
df.head()

,apply_at,apply_cnt,conv1_cnt,conv10_cnt
0,2023-01-01,72,5.0,28.0
1,2023-01-02,67,5.0,21.0
2,2023-01-03,51,6.0,17.0
3,2023-01-04,89,9.0,33.0
4,2023-01-05,67,6.0,18.0


In [102]:
df["conv10_ratio"] = df.conv10_cnt / df.apply_cnt
df["conv1_ratio"] = df.conv1_cnt / df.apply_cnt
df.head()

,apply_at,apply_cnt,conv1_cnt,conv10_cnt,conv10_ratio,conv1_ratio
0,2023-01-01,72,5.0,28.0,0.388889,0.069444
1,2023-01-02,67,5.0,21.0,0.313433,0.074627
2,2023-01-03,51,6.0,17.0,0.333333,0.117647
3,2023-01-04,89,9.0,33.0,0.370787,0.101124
4,2023-01-05,67,6.0,18.0,0.268657,0.089552


In [111]:
fig = px.line(
    df
    , x = "apply_at"
    , y = ["conv1_ratio", "conv10_ratio"]
)
fig.add_vline(x="2023-01-27")

# conversion window 에 따라 데이터가 신뢰  가능한 구간 표시
fig.add_vrect(x1 = "2023-01-27", x0='2023-01-17', fillcolor='red', opacity=.2)
fig.add_vrect(x1 = "2023-01-27", x0='2023-01-26', fillcolor='blue', opacity=.2)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

